本章将介绍使用**深度学习模型处理文本（可以将其理解为单词序列或字符序列）、时间序列和一般的序列数据**。用于处理序列的两种基本的深度学习算法分别是：

* 循环神经网络（recurrentneural network）

* 一维卷积神经网络（1D convnet），是二维卷积神经网络的一维版本

这些算法的应用包括：

* 文档分类和时间序列分类，比如识别文章的主题或书的作者；

* 时间序列对比，比如估测两个文档或两支股票行情的相关程度；

* 序列到序列的学习，比如将英语翻译成法语；

* 情感分析，比如将推文或电影评论的情感划分为正面或负面；

* 时间序列预测，比如根据某地最近的天气数据来预测未来天气。

本章的示例重点讨论两个小任务：

* 一个是 IMDB 数据集的情感分析，这个任务前面介绍过；

* 另一个是温度预测。

但这两个任务中所使用的技术可以应用于上面列出来的所有应用。

* 文本是最常用的序列数据之一，可以理解为**字符序列或单词序列，但最常见的是单词级处理**。

* 深度学习序列处理模型都可以根据文本生成基本形式的自然语言理解，并可用于**文档分类、情感分析、作者识别甚至问答（QA，在有限的语境下）**等应用。

* 请记住，本章的这些**深度学习模型都没有像人类一样真正地理解文本，而只是映射出书面语言的统计结构，但这足以解决许多简单的文本任务**。

* 深度学习用于自然语言处理是**将模式识别应用于单词、句子和段落，这与计算机视觉是将模式识别应用于像素大致相同**。与其他所有神经网络一样，**深度学习模型不会接收原始文本作为输入，它只能处理数值张量**。

* **文本向量化（vectorize）是指将文本转换为数值张量的过程。它有多种实现方法。**

    * 将文本分割为单词，并将每个单词转换为一个向量。
    * 将文本分割为字符，并将每个字符转换为一个向量。
    * 提取单词或字符的 n-gram，并将每个 n-gram 转换为一个向量。 n-gram 是多个连续单词或字符的集合（n-gram 之间可重叠）。
    
将文本分解而成的单元（单词、字符或 n-gram）叫作**标记（token）**，将文本分解成标记的过程叫作**分词（tokenization）**。

* 所有文本向量化过程都是应用某种分词方案，然后将**数值向量与生成的标记相关联**。这些向量组合成序列张量，被输入到深度神经网络中。将**向量与标记**相关联的方法有很多种。本节将介绍两种主要方法：
* 对标记做 one-hot 编码（one-hotencoding）
* 标记嵌入［token embedding，通常只用于单词，叫作词嵌入（word embedding）］。

下面介绍如何使用这些方法，将**原始文本转换为可以输入到 Keras网络中的 Numpy 张量**。

![image.png](../img/06/理解n-gram.png)

![image.png](../img/06/理解词袋.png)

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# 1 单词和字符的 one-hot 编码

* **单词级的 one-hot 编码**：`one-hot`编码是将标记转换为向量的最常用、最基本的方法。它将每个单词与一个唯一的整数索引相关联，然后将这个整数索引 `i` 转换为长度为 `N` 的二进制向量（`N` 是词表大小），这个向量只有第 `i` 个元素是 `1`，其余元素都为 `0`。

* **字符级的 `one-hot` 编码**

In [3]:
# 单词级的 one-hot 编码

import numpy as np
# 初始数据：每个样本是列表的一个元素（本例中的样本是一个句子，但也可以是一整篇文档）
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 构建数据中所有标记的索引
token_index = {}
for sample in samples:
    # 利用 split 方法对样本进行分词。在实际应用中，还需要从样本中去掉标点和特殊字符
    for word in sample.split():
        if word not in token_index:
            # 为每个唯一单词指定一个唯一索引
            token_index[word] = len(token_index) + 1
            # 注意，没有为索引编号 0 指定单词

# 对样本进行分词。只考虑每个样本前 max_length 个单词
max_length = 10

# 将结果保存在 results 中
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

In [4]:
# 字符级的 one-hot 编码

import string
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # 所有可打印的 ASCII 字符
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.
results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

注意， Keras 的内置函数可以对原始文本数据进行单词级或字符级的 one-hot 编码。你应该使用这些函数，因为它们实现了许多重要的特性，比如从字符串中去除特殊字符、只考虑数据集中前 N 个最常见的单词（这是一种常用的限制，以避免处理非常大的输入向量空间）。

## 用 Keras 实现单词级的 one-hot 编码

In [5]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 创建一个分词器（tokenizer），设置为只考虑前1000个最常见的单词
tokenizer = Tokenizer(num_words=1000)

# 构建单词索引
tokenizer.fit_on_texts(samples)

# 将字符串转换为整数索引组成的列表
sequences = tokenizer.texts_to_sequences(samples)

# 也可以直接得到 one-hot二进制表示。这个分词器也支持除 one-hot 编码外的其他向量化模式
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# 找回单词索引
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


one-hot 编码的一种变体是所谓的 one-hot 散列技巧（one-hot hashing trick），如果词表中唯一标记的数量太大而无法直接处理，就可以使用这种技巧。这种方法没有为每个单词显式分配一个索引并将这些索引保存在一个字典中，而是将单词散列编码为固定长度的向量，通常用一个非常简单的散列函数来实现。这种方法的主要优点在于，**它避免了维护一个显式的单词索引，从而节省内存并允许数据的在线编码（在读取完所有数据之前，你就可以立刻生成标记向量）**。这种方法有一个缺点，就是**可能会出现散列冲突（hash collision），即两个不同的单词可能具有相同的散列值，随后任何机器学习模型观察这些散列值，都无法区分它们所对应的单词**。如果散列空间的维度远大于需要散列的唯一标记的个数，散列冲突的可能性会减小。

# 使用散列技巧的单词级的 one-hot 编码

In [7]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 将单词保存为长度为 1000 的向量。如果单词数量接近 1000 个（或更多），那么会遇到很多散列冲突，这会降低这种编码方法的准确性
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # 将单词散列为 0~1000 范围内的一个随机整数索引
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.
results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

# 2 使用词嵌入(重要重要重要)

将单词与向量相关联还有另一种常用的强大方法，就是使用密集的**词向量（word vector）**，也叫**词嵌入（word embedding）**。 

* **`one-hot`编码**得到的向量是**二进制的、稀疏的（绝大部分元素都是`0`）、维度很高的（维度大小等于词表中的单词个数）**

* **词嵌入**是低维的浮点数向量（即密集向量，与稀疏向量相对）。

* 与 `one-hot` 编码得到的词向量不同，词嵌入是从数据中学习得到的。常见的词向量维度是 `256`、`512`或`1024`（处理非常大的词表时）。与此相对，`onehot`编码的词向量维度通常为`20000`或更高（对应包含`20000`个标记的词表）。因此，词向量可以将更多的信息塞入更低的维度中。

![image.png](../img/06/one-hot编码和词嵌入区别.png)

获取**词嵌入**有两种方法。

* 在**完成主任务（比如文档分类或情感预测）的同时学习词嵌入**。在这种情况下，一开始是**随机的词向量，然后对这些词向量进行学习，其学习方式与学习神经网络的权重相同**。

* **在不同于待解决问题的机器学习任务上预计算好词嵌入，然后将其加载到模型中**。这些**词嵌入**叫作**预训练词嵌入（pretrained word embedding）**。

## 2.1  利用 Embedding 层学习词嵌入

要将一个词与一个密集向量相关联，最简单的方法就是随机选择向量。这种方法的问题在于，得到的嵌入空间没有任何结构。例如， `accurate` 和 `exact` 两个词的嵌入可能完全不同，尽管它们在大多数句子里都是可以互换的。深度神经网络很难对这种杂乱的、非结构化的嵌入空间进行学习。

说得更抽象一点，**词向量之间的几何关系应该表示这些词之间的语义关系**。
* **词嵌入的作用应该是将人类的语言映射到几何空间中**。
* 在一个合理的嵌入空间中，同义词应该被嵌入到相似的词向量中，一般来说，任意两个词向量之间的几何距离（比如 L2 距离）应该和这两个词的语义距离有关（表示不同事物的词被嵌入到相隔很远的点，而相关的词则更加靠近）。
* **除了距离，你可能还希望嵌入空间中的特定方向也是有意义的**

![image.png](../img/06/词嵌入空间的简单示例.png)

在图中，四个词被嵌入在二维平面上，这四个词分别是 `cat`（猫）、 `dog`（狗）、 `wolf`（狼）和 `tiger`（虎）。对于我们这里选择的向量表示，这些词之间的某些语义关系可以被编码为几何变换。例如，从 `cat` 到 `tiger` 的向量与从 `dog` 到 `wolf` 的向量相等，这个向量可以被解释为“从宠物到野生动物”向量。同样，从 `dog` 到 `cat` 的向量与从 `wolf` 到 `tiger` 的向量也相等，它可以被解释为“从犬科到猫科”向量。

在真实的词嵌入空间中，**常见的有意义的几何变换的例子包括“性别”向量和“复数”向量**。例如，将 `king`（国王）向量加上 `female`（女性）向量，得到的是 `queen`（女王）向量。将 `king`（国王）向量加上 `plural`（复数）向量，得到的是 `kings` 向量。词嵌入空间通常具有几千个这种可解释的、并且可能很有用的向量。

**有没有一个理想的词嵌入空间，可以完美地映射人类语言，并可用于所有自然语言处理任务？**可能有，但我们尚未发现。此外，也不存在**人类语言（`human language`）**这种东西。**世界上有许多种不同的语言，而且它们不是同构的，因为语言是特定文化和特定环境的反射。但从更实际的角度来说，一个好的词嵌入空间在很大程度上取决于你的任务**。英语电影评论情感分析模型的完美词嵌入空间，可能不同于英语法律文档分类模型的完美词嵌入空间，因为某些语义关系的重要性因任务而异。

因此，**合理的做法是对每个新任务都学习一个新的嵌入空间**。幸运的是，**反向传播让这种学习变得很简单，而`Keras` 使其变得更简单。我们要做的就是学习一个层的权重，这个层就是`Embedding`层**。

In [8]:
# 将一个 Embedding 层实例化

from keras.layers import Embedding
# Embedding 层至少需要两个参数：标记的个数（这里是 1000，即最大单词索引 +1）和嵌入的维度（这里是 64）
embedding_layer = Embedding(1000, 64)

最好将 `Embedding` 层理解为一个字典，将**整数索引**（表示特定单词）映射为**密集向量**。它接收整数作为输入，并在内部字典中查找这些整数，然后返回相关联的向量。 `Embedding` 层实际上是一种字典查找。

**َ单词索引——>Embedding层——>对应的词向量**

![image.png](../img/06/Embedding层.png)

In [9]:
# 加载IMDB数据，准备用于Embedding层

from keras.datasets import imdb
from keras import preprocessing

# 作为特征的单词个数
max_features = 10000

# 在这么多单词后截断文本（这些单词都属于前 max_features 个最常见的单词）
maxlen = 20

# 将数据加载为整数列表
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 将整数列表转换成形状为(samples,maxlen) 的二维整数张量
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [10]:
# 在 IMDB 数据上使用 Embedding 层和分类器

from keras.models import Sequential
from keras.layers import Flatten, Dense


# 指定 Embedding 层的最大输入长度，以便后面将嵌入输入展平。 Embedding 层激活的形状为 (samples, maxlen, 8)
model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))

# 将三维的嵌入张量展平成形状为 (samples, maxlen * 8) 的二维张量
model.add(Flatten())

# 在上面添加分类器
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 12s 597us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 7s 358us/step - loss: 0.5657 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 6s 316us/step - lo

得到的验证精度约为 `76%`，考虑到仅查看每条评论的前 `20` 个单词，这个结果还是相当不错的。但请注意，仅仅将嵌入序列展开并在上面训练一个 `Dense` 层，会导致模型对输入序列中的每个单词单独处理，而没有考虑单词之间的关系和句子结构（举个例子，这个模型可能会将 `this movie is a bomb` 和 `this movie is the bomb` 两条都归为负面评论。更好的做法是**在嵌入序列上添加循环层或一维卷积层，将每个序列作为整体来学习特征**。这也是接下来几节的重点。

## 2.2 使用预训练的词嵌入

有时可用的训练数据很少，以至于只用手头数据无法学习适合特定任务的词嵌入。那么应该怎么办？

你可以从预计算的嵌入空间中加载嵌入向量（你知道这个嵌入空间是高度结构化的，并且具有有用的属性，即抓住了语言结构的一般特点），而不是在解决问题的同时学习词嵌入。在自然语言处理中使用预训练的词嵌入，其背后的原理与在图像分类中使用预训练的卷积神经网络是一样的：**没有足够的数据来自己学习真正强大的特征，但你需要的特征应该是非常通用的，比如常见的视觉特征或语义特征。在这种情况下，重复使用在其他问题上学到的特征，这种做法是有道理的。**

这种**词嵌入通常是利用词频统计计算得出的（观察哪些词共同出现在句子或文档中），用到的技术很多，有些涉及神经网络，有些则不涉及**。 **最有名且最成功的词嵌入方案之一 word2vec 算法发布之后，这一思路才开始在研究领域和工业应用中取得成功**。 word2vec 算法由 Google 的 Tomas Mikolov 于 2013 年开发，其维度抓住了特定的语义属性，比如性别。

* 有许多预计算的词嵌入数据库，你都可以下载并在 Keras 的 Embedding 层中使用。word2vec 就是其中之一。

* 另一个常用的是 GloVe（global vectors for word representation，词表示全局向量），由斯坦福大学的研究人员于 2014 年开发。这种嵌入方法基于对词共现统计矩阵进行因式分解。其开发者已经公开了数百万个英文标记的预计算嵌入，它们都是从维基百科数据和 Common Crawl 数据得到的。

我们来看一下如何在 Keras 模型中使用 GloVe 嵌入。同样的方法也适用于 word2vec 嵌入或其他词嵌入数据库。这个例子还可以改进前面刚刚介绍过的文本分词技术，即从原始文本开始，一步步进行处理。 

# 3 整合在一起：从原始文本到词嵌入

将句子嵌入到向量序列中，然后将其展平，最后在上面训练一个 Dense 层。但此处将使用预训练的词嵌入。

此外，我们将从头开始，先下载IMDB 原始文本数据，而不是使用 Keras 内置的已经预先分词的 IMDB 数据。

## 3.1 下载 IMDB 数据的原始文本

首先，打开 http://mng.bz/0tIo  下载原始 IMDB 数据集并解压。
接下来，我们将训练评论转换成字符串列表，每个字符串对应一条评论。
你也可以将评论标签（正面 / 负面）转换成 labels 列表。

In [17]:
# 处理 IMDB 原始数据的标签

import os

imdb_dir = r'D:\git\deep-learning-with-python\data\aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname),encoding='UTF-8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

## 3.2 对数据进行分词

利用本节前面介绍过的概念，**我们对文本进行分词，并将其划分为训练集和验证集**。因为预训练的词嵌入对训练数据很少的问题特别有用（否则，针对于具体任务的嵌入可能效果更好），所以我们又添加了以下限制：**将训练数据限定为前 200 个样本**。因此，你需要在读取 200 个样本之后学习对电影评论进行分类。

In [18]:
# 对 IMDB 原始数据的文本进行分词

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100                # 在 100 个单词后截断评论
training_samples = 200      # 在 200 个样本上训练
validation_samples = 10000  # 在 10 000 个样本上验证
max_words = 10000           # 只考虑数据集中前 10 000 个最常见的单词

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# 将数据划分为训练集和验证集，但首先要打乱数据，因为一开始数据中的样本是排好序的（所有负面评论都在前面，然后是所有正面评论）
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


## 3.3 下载 GloVe 词嵌入

打开 https://nlp.stanford.edu/projects/glove 下载 2014 年英文维基百科的预计算嵌入。这是一个 822 MB 的压缩文件，文件名是 glove.6B.zip，里面包含 400 000 个单词（或非单词的标记）的 100 维嵌入向量。解压文件。

## 3.4 对嵌入进行预处理

我们对解压后的文件（一个 .txt 文件）进行解析，构建一个将单词（字符串）映射为其向量表示（数值向量）的索引

In [19]:
# 解析 GloVe 词嵌入文件

glove_dir = r'D:\git\deep-learning-with-python\data\glove.6B'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'),encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


构建一个可以加载到 `Embedding` 层中的嵌入矩阵。它必须是一个形状为`(max_words, embedding_dim)`的矩阵，对于单词索引（在分词时构建）中索引为 `i` 的单词，这个矩阵的元素 `i` 就是这个单词对应的 `embedding_dim` 维向量。注意，索引 `0` 不应该代表任何单词或标记，它只是一个占位符。

In [20]:
# 准备 GloVe 词嵌入矩阵

embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # 嵌入索引（embeddings_index）中找不到的词，其嵌入向量全为 0
            embedding_matrix[i] = embedding_vector

## 3.5 定义模型

使用与前面相同的模型架构

In [21]:
# 模型定义

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


## 3.6 在模型中加载 GloVe 嵌入

`Embedding`层只有一个权重矩阵，是一个二维的浮点数矩阵，其中每个元素 `i` 是与索引 `i` 相关联的词向量。将准备好的 `GloVe` 矩阵加载到 `Embedding` 层中，即模型的第一层。

In [22]:
# 将预训练的词嵌入加载到 Embedding 层中

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

此外，需要冻结 `Embedding` 层（即将其 `trainable` 属性设为 `False`），其原理和预训练的卷积神经网络特征相同，你已经很熟悉了。如果一个模型的一部分是经过预训练的（如 `Embedding` 层），而另一部分是随机初始化的（如分类器），那么在训练期间不应该更新预训练的部分，以避免丢失它们所保存的信息。随机初始化的层会引起较大的梯度更新，会破坏已经学到的特征。

## 3.7 训练模型与评估模型

In [ ]:
# 训练模型与评估模型

# 训练与评估

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 2s 8ms/step - loss: 1.6337 - acc: 0.5250 - val_loss: 0.7130 - val_acc: 0.5100
Epoch 2/10
200/200 [==============================] - 1s 5ms/step - loss: 0.7565 - acc: 0.5800 - val_loss: 0.6910 - val_acc: 0.5418
Epoch 3/10
192/200 [===========================>..] - ETA: 0s - loss: 0.5946 - acc: 0.6927

In [ ]:
# 绘制模型性能随时间的变化,绘制结果

%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# 在不使用预训练词嵌入的情况下，训练相同的模型
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

验证精度停留在 `50%` 多一点。因此，预训练词嵌入的性能要优于与任务一起学习的嵌入。如果增加样本数量，情况将很快发生变化，你可以把它作为一个练习。

**在测试数据上评估模型。首先，你需要对测试数据进行分词。**

In [ ]:
# 对测试集数据进行分词

test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname),encoding='UTF-8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [ ]:
# 在测试集上评估模型

model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

测试精度达到了令人震惊的 56% ！只用了很少的训练样本，得到这样的结果很不容易。

* 将原始文本转换为神经网络能够处理的格式。
* 使用 Keras 模型的 Embedding 层来学习针对特定任务的标记嵌入。
* 使用预训练词嵌入在小型自然语言处理问题上获得额外的性能提升。